In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

In [17]:
# Загрузка обучающего набора данных
train_data = pd.read_csv('train.csv')
X = train_data[['x', 'y', 'z']]
y = train_data[['u_x', 'u_y', 'u_z', 'p']]

# Загрузка тестового набора данных
test_data = pd.read_csv('test.csv')






In [31]:
# Параметры для Random Search
params = {
    'n_estimators': [1000, 10000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 10]
}

# Словарь для хранения лучших моделей для каждой целевой переменной
best_models = {}


In [32]:
for target in y.columns:
    print(f"Optimizing for {target}")
    xgb = XGBRegressor(random_state=42)
    random_search = GridSearchCV(xgb, param_grid=params, scoring='neg_mean_absolute_error', n_jobs=-1, cv=3)
    random_search.fit(X, y[target])
    best_models[target] = random_search.best_estimator_
    print(- 1.0* random_search.best_score_)

Optimizing for u_x


1.0455240271752064
Optimizing for u_y
1.0742307210112092
Optimizing for u_z
1.0478771883052072
Optimizing for p
492.62152519969555


In [33]:
# Делаем предсказания для тестового набора данных и сохраняем результаты
predictions = {}
for target, model in best_models.items():
    predictions[target] = model.predict(test_data)

In [ ]:
# Сохраняем предсказания в DataFrame и в файл CSV
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('predictions.csv', index=False)